# Build the Neural Network
https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Define the Class

In [6]:
# PyTorch networks are defined by subclassing `nn.Module`
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        # Every nn.Module subclass implements the operations on input data in the `forward` method
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
# We need to create an instance of NeuralNetwork, and move it to the `device`, and print its structure
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
# To use the model, we pass it the input data. This executes the model's `forward`, along with some background operations. DO NOT CLASS `model.forward()` DIRECTLY!
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


## Model Layers

In [12]:
input_image = torch.rand(3, 28, 28)
print(input_image.size()) # Same as input_image.shape

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained)

In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The `linear layer` is a module that applies a linear transformation on the input using its stored weights and biases. (I think it is the same as Dense layer in TensorFlow)

In [14]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLu
Non-linear activations are what create the complex mappings between the model's inputs and outputs. They are applied after linear transformations to introduce nonlinearity, hepling neural networks learn a wide variety of phenomena.

In this model, we use `nn.ReLu` between our linear layers, but there's other activations to introduce non-linearity in your model.

In [15]:
print(f"Before ReLu: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLu: tensor([[ 0.0015,  0.4725, -0.1556,  0.1000, -0.2319,  0.2583, -0.1844,  0.7144,
          0.0709,  0.7565,  0.2777,  0.1006, -0.2272,  0.3192, -0.2666, -0.5671,
          0.3815,  0.2911,  0.4225,  0.4485],
        [ 0.1559,  0.2325, -0.4059, -0.1381,  0.1776,  0.1807, -0.1633,  0.8494,
          0.3910,  0.7399, -0.0390, -0.0426,  0.0373,  0.5506, -0.3072, -0.4336,
          0.0163, -0.0978,  0.4908,  0.3832],
        [ 0.1031,  0.3739,  0.1862,  0.1278, -0.2375,  0.2948, -0.1271,  0.6053,
         -0.0962,  0.6935, -0.1288,  0.1346,  0.0280, -0.0835, -0.3107, -0.4693,
          0.0804,  0.2733,  0.2636,  0.6471]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0015, 0.4725, 0.0000, 0.1000, 0.0000, 0.2583, 0.0000, 0.7144, 0.0709,
         0.7565, 0.2777, 0.1006, 0.0000, 0.3192, 0.0000, 0.0000, 0.3815, 0.2911,
         0.4225, 0.4485],
        [0.1559, 0.2325, 0.0000, 0.0000, 0.1776, 0.1807, 0.0000, 0.8494, 0.3910,
         0.7399, 0.0000, 0.0000, 0.0373, 0.5506, 0.00

### nn.Sequential
`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. We can use sequential containers to put together a quick network like seq_modules.

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns logits - raw values in [-Infinity, Infinity] - which are passed to the `nn.Softmax` module. The logits are scaled to values [0, 1], representing the model's predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0996, 0.0913, 0.0842, 0.1049, 0.1112, 0.1222, 0.1011, 0.0878, 0.0989,
         0.0988],
        [0.0940, 0.0979, 0.0776, 0.1078, 0.1086, 0.1198, 0.1081, 0.0934, 0.0913,
         0.1015],
        [0.0963, 0.0968, 0.0695, 0.1144, 0.0928, 0.1323, 0.1098, 0.1023, 0.0879,
         0.0979]], grad_fn=<SoftmaxBackward0>)

## Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimizer during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [19]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0021,  0.0132,  0.0164,  ..., -0.0067,  0.0095,  0.0306],
        [-0.0297,  0.0014,  0.0021,  ..., -0.0027,  0.0227,  0.0177]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0255,  0.0098], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0193,  0.0434,  0.0431,  ...,  0.0294, -0.0331,  0.0020],
        [ 0.0142,  0.0257, -0.0108,  ...,  0.0164, -0.0149, -0.0213]],
       device='cuda:0', grad_fn=<Slice